# FEniCS Example

This notebook outlines how to use [FEniCS](https://fenicsproject.org/) to solve the ``OnceClampedBeam`` problem from Section 3.1 of [this paper](https://www.sciencedirect.com/science/article/pii/S0045782523004759). 

In brief, for this problem we have a rectangular beam which follows the Neo-Hookean constitutive law. The beam is clamped at one end, and we wish to solve for the displacement of the beam as it deforms under its own weight due to gravity.

##### Import packages

In [1]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt
import os

FEniCS can be installed by uncommenting the below:

In [2]:
# ! conda install -c conda-forge fenics

## Set up problem

##### Specify material parameter values $\boldsymbol{\theta}$

In [3]:
theta = np.array([5e5, 1e6])

n_params = theta.shape[0]

##### Create mesh and define function space

In [4]:
# finite element mesh
mesh = BoxMesh(Point(0.0, 0.0, 0.0), Point(10., 1., 1.), 12, 4, 5)

# define function space
V = VectorFunctionSpace(mesh, 'P', 1)

# check dimension of the function space
V.dim()

1170

##### Specify the clamped Dirichlet boundary conditions

In [5]:
# boundary is left hand side of the beam
left =  CompiledSubDomain("near(x[0], side) && on_boundary", side = 0.0)

# no displacement on boundary
c = Constant((0.0, 0.0, 0.0))

bcl = DirichletBC(V, c, left)
bcs = [bcl]

##### Define test and trial functions

In [6]:
# Incremental displacement
du = TrialFunction(V)            

# Test function
v  = TestFunction(V)             

# Displacement from previous iteration
u  = Function(V)                 

##### Define external body force (gravity)

In [7]:
# density of the beam
rho = 1.

# acceleration due to gravity
g   = 980. 

# Body force per unit volume
B = Constant((0., 0., -rho*g))

##### Define total potential energy $\Pi$

In [8]:
def compute_Pi(u, material_params, return_all_values=False):
    
    # dimensionality of the system
    D = len(u)
    
    # identity matrix
    I = Identity(D)
    
    # Deformation gradient
    F = I + grad(u)  
    
    # Right Cauchy-Green tensor
    C = F.T*F        
    
    # Invariants
    Ic = tr(C)
    J  = det(F)
    
    # extract Lame parameters
    mu, lambda_ = material_params

    # compressible Neo-Hookean constitutive law
    psi = (mu/2)*(Ic - 3) - mu*ln(J) + (lambda_/2)*(ln(J))**2

    # internal potential energy is the integral of psi over the domain
    Pi_internal= psi*dx
    
    # external potential energy for this example is the work due to body force B
    Pi_external = dot(B, u)*dx
    
    # Total potential energy 
    Pi = Pi_internal - Pi_external

    # we might want to inspect internal and external potential energies
    if return_all_values:
        return Pi, Pi_internal, Pi_external

    # otherwise just return Pi
    return Pi

Pi = compute_Pi(u, theta)

## Solve Problem

In [9]:
# Compute first variation of Pi (directional derivative about u in the direction of v)
G = derivative(Pi, u, v)

# Compute Jacobian of F
dG = derivative(G, u, du)

# Solve variational problem
solve(G == 0, u, bcs, J=dG)

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 5.442e+02 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.018e+06 (tol = 1.000e-10) r (rel) = 1.870e+03 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 3.543e+05 (tol = 1.000e-10) r (rel) = 6.511e+02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 3.831e+04 (tol = 1.000e-10) r (rel) = 7.040e+01 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 2.744e+03 (tol = 1.000e-10) r (rel) = 5.042e+00 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 4.267e+02 (tol = 1.000e-10) r (rel) = 7.841e-01 (tol = 1.000e-09)
  Newton iteration 6: r (abs) = 9.444e+00 (tol = 1.000e-10) r (rel) = 1.735e-02 (tol = 1.000e-09)
  Newton iteration 7: r (abs) = 3.041e-02 (tol = 1.000e-10) r (rel) = 5.587e-05 (tol = 1.000e-09)
  Newton iteration 8: r (abs) = 6.766e-08 (tol = 1.000e-10) r (rel) = 1.243e-10 (tol = 1.000e-09)
  Newton solver finished in 8 iterations and 8 linear solver iterations.


##### Extract results to numpy

In [10]:
displacement = u.compute_vertex_values().reshape(mesh.coordinates().shape, order='F')

displacement.shape

(390, 3)

##### Print statistics relating to the magnitude of the displacements

In [11]:
displacement_norms = ((displacement**2).sum(-1)**.5)
L, H, W = mesh.coordinates().max(0)
n_nodes = mesh.coordinates().shape[0]

print(f'Beam Dimensions    : {L}*{H}*{W} cm')
print(f'Num Nodes          : {n_nodes}')
print(f'mu / lambda        : {theta[0]}/{theta[1]}')
print(f'Max  Disp. Norm    : {displacement_norms.max():5f} cm')
print(f'Mean Disp. Norm    : {displacement_norms.mean():5f} cm\n')
print(f'displacement[:3]: \n{displacement[:3]}')

Beam Dimensions    : 10.0*1.0*1.0 cm
Num Nodes          : 390
mu / lambda        : 500000.0/1000000.0
Max  Disp. Norm    : 4.938831 cm
Mean Disp. Norm    : 2.042705 cm

displacement[:3]: 
[[ 0.          0.          0.        ]
 [-0.07520687 -0.00756748 -0.07513904]
 [-0.15358888 -0.00074147 -0.25225487]]


##### Compute the total potential energy value for the computed solution function

In [12]:
Pi_vals = compute_Pi(u, theta, True)
Pi, Pi_internal, Pi_external = [assemble(val) for val in Pi_vals]

In [13]:
print(f"Internal potential energy: {Pi_internal:2f}")
print(f"External potential energy: {Pi_external:2f}")
print(f"Total potential energy   : {Pi:2f}")

Internal potential energy: 8727.402580
External potential energy: 19003.827557
Total potential energy   : -10276.424978


## Saving results

The above code runs a simulation for a single value of the material parameters ``theta``.

To generate the test datasets used in the paper (like [here](https://github.com/dodaltuin/soft-tissue-pignn/tree/main/data/OnceClampedBeam/simulationData/test) here for example), I re-ran the fenics simulation code for different values of ``theta`` (this can be achieved by a simple for loop over the above commands).

I then collected the ``displacement``, `theta` and ``Pi_total`` values for each simulation into lists, concatenated them to numpy arrays and then saved the results.

The below code gives an idea of how this can be done:

In [14]:
# suppose for instance we have run 10 test simulations
n_test = 10

# the list of displacements / theta / Pi_total values would then look something like this: 
disp_list  = [displacement.copy()]*n_test
theta_list = [theta]*n_test
Pi_list    = [Pi]*n_test

Note that of course in practice, the elements of the above lists would differ because they would correspond to different simulations. Here they are all equal for simplicity - the code to save the results (see below) doesn't change.

The first thing to do is to concatenate into numpy arrays:

In [15]:
disp_arrs  = np.array(disp_list)
theta_arrs = np.array(theta_list)
Pi_arrs    = np.array(Pi_list)

The shape of ``disp_arrs`` is ``(n_test x n_nodes x D)``:

In [16]:
disp_arrs.shape

(10, 390, 3)

The shape of ``theta_arrs`` is ``(n_test x n_params)``:

In [17]:
theta_arrs.shape

(10, 2)

The shape of ``Pi_arrs`` is ``(n_test,)``:

In [18]:
Pi_arrs.shape

(10,)

Then create a directory to save the results in

In [19]:
results_save_dir = "exampleTestDir"
if not os.path.isdir(results_save_dir): os.makedirs(results_save_dir)
results_save_dir

'exampleTestDir'

The results can finally be saved:

In [20]:
np.save(f'{results_save_dir}/displacement.npy', disp_arrs)
np.save(f'{results_save_dir}/theta.npy', theta_arrs)
np.save(f'{results_save_dir}/pe-values.npy', Pi_arrs)

## Saving other results

Details on how to extract and save other results required for the graph neural network emulator (like the mesh topology) are provided in [this notebook here](https://github.com/dodaltuin/soft-tissue-pignn/blob/main/data/fenicsMeshProcessing/FenicsMeshProcessingLV.ipynb).